# Import packages

In [1]:
import os
import time 
import numpy as np
import pandas as pd
import geopandas as gp
import cftime 

In [2]:
from sklearn.linear_model import LinearRegression

In [3]:
import xarray as xr
import rioxarray
from rasterio.enums import Resampling

In [4]:
# Time the script
start_time = time.perf_counter ()

# Set up directories and file paths

In [5]:
path_data_in = '../2 Raw Data'
path_map = '../4 Mapping'
path_data_out = '../3 Output Data'

## Input data

In [6]:
path_pop_count = os.path.join(path_data_in, 
                              'Population', 
                              'Count',
                              'zaf_ppp_2015_1km_Aggregated.tif')  


dir_pop_agesex= os.path.join(path_data_in, 
                             'Population', 
                             'AgeSex')
#file_healthsiteIO_shp=os.path.join(path_data_in, 
                                 #  'Facilities',
                                  # 'Uzbekistan_HealthsitesIO_SHP',
                                  # "Uzbekistan-node.shp")
#file_healthsiteIO_csv=os.path.join(path_data_out,
                                #   'Uzbekistan_healthsites_io.csv')
path_gdp=os.path.join(path_data_in,
                      'GDP_HDI',
                      'GDP_PPP_30arcsec_v3.nc')

path_hotdays =os.path.join(path_map,
                      'anomaly-hd35-annual-mean_cmip6_annual_all-regridded-bct-ssp119-climatology_median_2020-2039.nc')


path_nl = os.path.join(path_data_in,
                       'Nighttime_Lights',
                       'zaf_viirs_100m_2015.tiff')
#no birth data for Albania
#path_birth=os.path.join(path_data_in,
 #                       'Birth',
  #                      'Uzbekistan_1km_Births',
   #                     'uzb_births_pp_v2_2015.tif')
# path_death = os.path.join(path_data_in,
#                           'Death',
#                           'Albania_death_data_by_age_and_sex_2019.csv') 

## Output data

In [7]:
#path_healthfacility_public = os.path.join(path_data_out,"public_health_facilities.csv")
#path_healthfacility_private = os.path.join(path_data_out,"private_health_facilities.csv")
path_agesex = os.path.join(path_data_out,"age_sex_distribution_1km_2019.csv")
#bank_csv_path = os.path.join(data_in_path,"Egypt_Banks_and_ Atms.csv")

# User-defined functions

In [8]:
def find_grid_index(x,y,longs,lats,x_min=None,x_max=None,y_min=None,y_max=None):
    """
    DESCRIPTION: This function helps to find the index of the longitudes and 
    latitudes in the given longitude/latitude lists that is closest to the 
    given point (x,y) 
    
    INPUT -
    x: latitude of the given point
    y: longtidue of the given point
    longs: reference list of longitudes
    lats: reference list of latitudes
    x_min, x_max: min/max boundaries of longitude
    y_min, y_max: min/max boundaries of latitude
    
    OUTPUT -
    Index of the closest longitude/latitude to point (x,y)
    """
    if x_min is None and x_max is None:
        x_l = longs[0]- (longs[1]-longs[0])
        x_r = longs[-1] + (longs[-1]-longs[-2])
        x_min = min(x_l, x_r)
        x_max = max(x_l, x_r)
    if x < x_min or x > x_max:
        return -1,-1
    
    if y_min is None and y_max is None:
        y_l = lats[0]- (lats[1]-lats[0])
        y_r = lats[-1] + (lats[-1]-lats[-2])
        y_min = min(y_l, y_r)
        y_max = max(y_l, y_r)
    if y < y_min or y > y_max:
        return -1,-1
    
    ncol = np.argmin(abs(longs-x)) 
    nrow = np.argmin(abs(lats-y))
    if nrow<0 or nrow>=(len(lats)-1) or ncol<0 or ncol>=(len(longs)-1):
        return -1, -1
    else:
        return nrow,ncol


def points_in_grid_new(points_df,ref_data,value_colname=None):
    """
    DESCRIPTION: This function finds how many points (and how many non nan points and their sum  
    if a value_colname is provided) lying in the grids centered at the coordinates of the reference data
    
    INPUT -
    points_df: pandas dataframe that needs to be count 
    ref_data: xarray DataArray with reference coordinates
    value_colname: column name for the values in the dataframe
    
    OUTPUT -
    count: How many points are in each grid of the reference coordinate
    non_nan_count: How many non nan points are in each grid of the reference coordinate
    value_sum: The sum of the values of the non nan points within each grid of the reference coordinate
    """
    long_list = ref_data.coords['x'].values
    lat_list = ref_data.coords['y'].values
    long_list = np.append(long_list, long_list[-1]+ref_data.rio.resolution()[0])
    lat_list = np.append(lat_list, lat_list[-1]+ref_data.rio.resolution()[1])
    count = np.zeros(ref_data.values.shape)
    non_nan_count = np.zeros(ref_data.values.shape)
    if value_colname is not None:
        value_sum = np.empty(ref_data.values.shape)
        value_sum[:] = np.nan
        values = points_df[value_colname]
    else:
        value_sum = None
    grid_index = points_df.apply(lambda row: find_grid_index(row['Long'],row['Lat'],long_list,lat_list), axis=1)
    for j, x in enumerate(grid_index):
        if x[0]!=-1 and x[1]!=-1:
            count[x[0],x[1]] += 1
            if value_colname is not None and ~np.isnan(values[j]):
                non_nan_count[x[0],x[1]] = non_nan_count[x[0],x[1]]+1 
                value_sum[x[0],x[1]] = np.nansum([value_sum[x[0],x[1]],values[j]])
    return count, non_nan_count, value_sum



def spatial_aggregate(count_in,value_sum_in,block_size,agg_mode='sum', non_nan_count_in=None):
    """
    DESCRIPTION: This function apply square sum aggregation with stride 1 for 2D numpy array
    with the same padding
    
    INPUT -
    count_in: 2D numpy array for the count at each grid
    value_sum_in: 2D numpy array for the sum of values at each grid
    block_size: the size of the square sum aggregation
    agg_mode: aggregation operation mode "sum" or "mean"
    non_nan_count_in: 2D array for the non nan count at each grid, only needed for "mean" mode
    
    
    OUTPUT - 
    a new 2D numpy array with square sum aggreation in the same size
    """
    if block_size%2 != 1:
        raise Exception('block_size must be an odd integer!')
    half = block_size//2

    new_count_in = np.zeros(count_in.shape)
    count_in = np.pad(count_in,half)

    return_value = np.zeros(value_sum_in.shape)
    value_sum_in = np.pad(value_sum_in,half)
    
    if agg_mode == 'mean':
        new_nonnan_count_in = np.zeros(non_nan_count_in.shape)
        non_nan_count_in = np.pad(non_nan_count_in,half)
    elif agg_mode == 'sum':
        new_nonnan_count_in = None
    else:
        raise Exception('Error: The function only supports agg_mode for sum or mean.')
    for nrow in range(half,count_in.shape[0]-half):
        for ncol in range(half,count_in.shape[1]-half):
            new_count_in[nrow-half,ncol-half] = np.sum(count_in[(nrow-half):(nrow+half+1),(ncol-half):(ncol+half+1)])
            if agg_mode == 'mean':
                new_nonnan_count_in[nrow-half,ncol-half] = np.sum(non_nan_count_in[(nrow-half):(nrow+half+1),(ncol-half):(ncol+half+1)])
            return_value[nrow-half,ncol-half] = np.nansum(value_sum_in[(nrow-half):(nrow+half+1),(ncol-half):(ncol+half+1)])
    if agg_mode == 'mean':
        new_nonnan_count_in = np.where(new_nonnan_count_in == 0, np.nan, new_nonnan_count_in)
        return_value = return_value/new_nonnan_count_in
        return new_count_in, return_value, new_nonnan_count_in
    else:
        return new_count_in, return_value, new_nonnan_count_in


def layer_aggregate_helper(count_in,value_sum_in,block_size,non_nan_count_in):
    """
    DESCRIPTION: This function apply square sum aggregation with stride 1 for 2D numpy array
    with the same padding
    
    INPUT -
    count_in: 2D numpy array for the count at each grid
    value_sum_in: 2D numpy array for the sum of values at each grid
    block_size: the size of the square sum aggregation
    non_nan_count_in: 2D array for the non nan count at each grid
    
    
    OUTPUT - 
    a new 2D numpy array with square sum aggreation in the same size
    """
    if block_size%2 != 1:
        raise Exception('block_size must be an odd integer!')
    half = block_size//2

    new_count_in = np.zeros(count_in.shape)
    count_in = np.pad(count_in,half)

    return_value = np.zeros(value_sum_in.shape)
    value_sum_in = np.pad(value_sum_in,half)
    
    new_nonnan_count_in = np.zeros(non_nan_count_in.shape)
    non_nan_count_in = np.pad(non_nan_count_in,half)
  
    for nrow in range(half,count_in.shape[0]-half):
        for ncol in range(half,count_in.shape[1]-half):
            new_count_in[nrow-half,ncol-half] = np.sum(count_in[(nrow-half):(nrow+half+1),(ncol-half):(ncol+half+1)])
            new_nonnan_count_in[nrow-half,ncol-half] = np.sum(non_nan_count_in[(nrow-half):(nrow+half+1),(ncol-half):(ncol+half+1)])
            return_value[nrow-half,ncol-half] = np.nansum(value_sum_in[(nrow-half):(nrow+half+1),(ncol-half):(ncol+half+1)])
    return_value = np.where(new_nonnan_count_in == 0, np.nan, return_value)
    return new_count_in, return_value, new_nonnan_count_in



def layer_aggregate(value1,count1,value2,count2,mode = 'sum'):
    value1 = np.array(value1)
    value2 = np.array(value2)
    count1 = np.array(count1)
    count2 = np.array(count2)
    value2[np.where(count1==count2)]=value1[np.where(count1==count2)]
    value1 = np.where(np.isnan(value1), 0, value1)
    value2 = np.where(np.isnan(value2), 0, value2)
    value_diff = value2-value1
    count_diff = count2-count1
    if mode == 'sum':
        return_value = value_diff
        return_value = np.where(count_diff==0,np.nan,return_value)
    elif mode == 'mean':
        count_diff = np.where(count_diff == 0, np.nan, count_diff)
        return_value = value_diff/count_diff
    else:
        raise Exception('Only support sum or mean for mode')
    return return_value



def square_helper(arr, y, x, mode):
    """
    DESCRIPTION: This is a helper funtion to do mean or sum in square for 2D numpy array
    
    INPUT - 
    arr: 2D numpy array to be averaged
    (y,x): output dimension
    mode: sum or mean, NAs are ignored 

    OUTPUT - 
    Averaged 2D array
    """
    yy, xx = arr.shape
    if mode == "sum":
        vals = np.nansum(arr.reshape(y, yy//y, x, xx//x),(1,3))
    elif mode == "mean":
        vals = np.nanmean(arr.reshape(y, yy//y, x, xx//x),(1,3))
    else:
        raise Exception("Mode is not suppported, please input `sum' or `mean'")
    return vals



def xarray_square_aggregate(df_raw,stride=10,var_name="band_data",mode="mean"):
    """
    DESCRITPION: This is a function to do square aggregation (mean or sum) for a
    xarray dataArray, generating a new xarray dataArray
    
    INPUT -
    df_raw: 2D xarray DataArray
    stride: Size of the non-overlap square aggregation
    var_name: new DataArray variable name
    mode: square aggregation function
    
    OUTPU -
    A new 2D xarray DataArray with non-overlap square aggregation
    """
    nrow, ncol = df_raw.values.squeeze().shape
    new_nrow,new_ncol = int(nrow/stride), int(ncol/stride)
    data = df_raw.values.squeeze()[0:(new_nrow*stride),0:(new_ncol*stride)]
    new_data = square_helper(data, new_nrow, new_ncol, mode)
    new_x = np.mean(df_raw.coords['x'].values[0:(new_ncol*stride)].reshape(-1,stride),axis=1)
    new_y = np.mean(df_raw.coords['y'].values[0:(new_nrow*stride)].reshape(-1,stride),axis=1)
    # create new datarray
    df = xr.DataArray(
        data=new_data,
        dims=["y", "x"],
        coords=dict(
            x= new_x,
            y= new_y,
        ),
    )
    df.name = var_name
    if df_raw.rio.crs is not None:
        df = df.rio.write_crs(df_raw.rio.crs)
    return df



def grid_reproject(df,ref_data,resample_method = Resampling.nearest):
    """
    DESCRIPTION: This is a function to reproject a xarray dataArray to the coordinates
    of another refrence xarray dataArray.
    
    INPUT - 
    df: The xarray DataArray needing to be reprojected to the new grids
    ref_data: The reference xarray DataArray with reference coordinates (the new grids)
    resample_method: resample method
    
    OUTPUT -
    A new xarray DataArray reprojected to the coordinates of ref_data 
    """
    df = df.rio.reproject_match(ref_data, resampling = Resampling.nearest, nodata= np.nan)
    df = df.assign_coords({
        "x": ref_data.x,
        "y": ref_data.y,
    })
    return df



def points_in_grid(points_df,ref_data):
    """
    DESCRIPTION: This function finds how many points are in the grids centered 
    at the coordinates of the reference data
    
    INPUT -
    points_df: xarray DataArray that needs to be count 
    ref_data: xarray DataArray with reference coordinates
    
    OUTPUT -
    How many points are in each grid of the reference coordinates
    """
    long_list = ref_data.coords['x'].values
    lat_list = ref_data.coords['y'].values
    long_list = np.append(long_list, long_list[-1]+ref_data.rio.resolution()[0])
    lat_list = np.append(lat_list, lat_list[-1]+ref_data.rio.resolution()[1])
    count = np.zeros(ref_data.values.shape)
    grid_index = points_df.apply(lambda row: find_grid_index(row['Long'],row['Lat'],long_list,lat_list), axis=1)
    for x in grid_index:
        if x[0]!=-1 and x[1]!=-1:
            count[x[0],x[1]] += 1
    return count



def aggregate_count(count,block_size):
    """
    DESCRIPTION: This function apply square sum aggregation with sride 1 for 2D numpy array
    with same padding
    
    INPUT -
    count: 2D numpy array
    block_size: the size of the square sum aggregation
    
    OUTPUT - 
    a new 2D numpy array with square sum aggreation in the same size
    """
    if block_size%2 != 1:
        raise Exception('block_size must be an odd integer!')
    half = block_size//2
    new_count = np.zeros(count.shape)
    count = np.pad(count,half)
    for nrow in range(half,count.shape[0]-half):
        for ncol in range(half,count.shape[1]-half):
            new_count[nrow-half,ncol-half] = np.sum(count[(nrow-half):(nrow+half+1),(ncol-half):(ncol+half+1)])
    return new_count


    
def print_summary(data):
    """
    DESCRIPTION: This is a function to print the sumamry of a xarray DataArray or Dataset
    
    INPUT - 
    data: xarray DataArray or Dataset
    
    OUTPUT - 
    summary statistics of data
    """
    print(f"shape: {data.rio.shape}")
    print(f"resolution: {data.rio.resolution()}")
    print(f"coordinates boundary: {data.rio.bounds()}")
    print(f"CRS: {data.rio.crs}")

# Specify reference coordinates/grids

1. The spatial distribution of population in 2015 South Africa
2. zaf_ppp_2015_1km_Aggregated.tif
3. https://hub.worldpop.org/geodata/summary?id=33887

Here we use the grids of Uzbekistan population data as the reference data.
All other data will be reprojected to this reference grids to make all data consistent in space.

# Data Preprocessing

Process age and gender population data

1. Albania 100m Age and Gender structures in 2020
2.
3. https://www.worldpop.org/geodata/summary?id=50113

We have one Geotiff file for each sex class and each age bin, here we aggregate the 36 Geotiff files into one clean csv file, which will be used to calculate death data at each grid later.

### Set sex bins and age bins 

In [9]:
# sex_bins = ['Female','Male']
# age_bins = ['Age <1 year', 'Age 1 to 4','Age 5 to 9',
#             'Age 10 to 14','Age 15 to 19','Age 20 to 24',
#             'Age 25 to 29','Age 30 to 34','Age 35 to 39',
#             'Age 40 to 44','Age 45 to 49','Age 50 to 54',
#             'Age 55 to 59','Age 60 to 64','Age 65 to 69',
#             'Age 70 to 74','Age 75 to 79','Age 80 plus']
# age_start_bins = [0,1] + list(np.arange(5,80+1,5))

In [10]:
# %%time
# age_sex_df = None
# for sex in sex_bins:
#     for file_no, age in enumerate(age_bins):
#         file_name ="alb_"+sex[0].lower()+"_"+str(age_start_bins[file_no])+"_2020_constrained.tif"
#         col_name = sex + ' ' + age
#         age_sex_raw = xr.open_dataarray(os.path.join(dir_pop_agesex,file_name))
#         age_sex = xarray_square_aggregate(age_sex_raw,stride=10,var_name=col_name, mode="sum")
#         age_sex = grid_reproject(age_sex, pop)
#         df = age_sex.to_dataframe()
#         df = df.drop("spatial_ref",axis=1)
#         age_sex_df = df if age_sex_df is None else age_sex_df.merge(df, left_index=True, right_index=True) 
#         print(col_name)

### Rename and take a look at age and sex distribution

In [11]:
# %%time
# age_sex_df = age_sex_df.reset_index().rename({'x':'Long','y':'Lat'},
#                                              axis=1)
# age_sex_df

In [12]:
# %%time
# age_sex_df.to_csv(path_agesex)

## Number of hot days --- coordinate reference

In [13]:
hotdays_raw = rioxarray.open_rasterio(path_hotdays)[0]
display(hotdays_raw)
# 0: mean; 1: longitude boundary; 2: latitude boundary
print_summary(hotdays_raw)


/Users/yangshining/opt/miniconda3/lib/python3.9/site-packages/rasterio/__init__.py:220: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/Users/yangshining/opt/miniconda3/lib/python3.9/site-packages/rioxarray/_io.py:851: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  warnings.warn(str(rio_warning.message), type(rio_warning.message))  # type: ignore
/Users/yangshining/opt/miniconda3/lib/python3.9/site-packages/rasterio/__init__.py:220: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/Users/yangshining/opt/miniconda3/lib/python3.9/site-packages/rioxarray/_io.py:851: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  warnings.warn(str(rio_war

<xarray.Dataset>
Dimensions:                   (time: 1, x: 361, y: 181)
Coordinates:
  * time                      (time) object 2020-01-01 12:00:00
  * x                         (x) float64 -180.0 -179.0 -178.0 ... 179.0 180.0
  * y                         (y) float64 90.0 89.0 88.0 ... -88.0 -89.0 -90.0
    spatial_ref               int64 0
Data variables:
    anomaly-hd35-annual-mean  (time, y, x) float32 ...
Attributes: (12/30)
    activity_id:           ScenarioMIP
    climate_data_id:       CMIP6
    cmip6-ng:              Some daily data has been acquired through the CMIP...
    cmor_version:          3.4.0
    Conventions:           CF-1.7 CMIP-6.2
    creation_date:         2021-11-05T01:00:00Z
    ...                    ...
    parent_activity_id:    CMIP and ScenarioMIP
    parent_experiment_id:  piControl
    parent_mip_era:        CMIP6
    parent_time_units:     days since 1850-01-01
    realm:                 atmos
    table_id:              day

shape: (181, 361)
resolution: (1.0, -1.0)
coordinates boundary: (-180.5, -90.5, 180.5, 90.5)
CRS: None


In [14]:
# rename variable
hotdays_raw["hotdays"] = hotdays_raw["anomaly-hd35-annual-mean"]
hotdays_raw = hotdays_raw.drop(["anomaly-hd35-annual-mean"])

In [15]:
hotdays = hotdays_raw.isel(time=0)
hotdays = hotdays.reset_coords(names=['time'],drop=True)
hotdays = hotdays_raw.rio.write_crs("epsg:4326", inplace=True)
hotdays

<xarray.Dataset>
Dimensions:      (time: 1, x: 361, y: 181)
Coordinates:
  * time         (time) object 2020-01-01 12:00:00
  * x            (x) float64 -180.0 -179.0 -178.0 -177.0 ... 178.0 179.0 180.0
  * y            (y) float64 90.0 89.0 88.0 87.0 ... -87.0 -88.0 -89.0 -90.0
    spatial_ref  int64 0
Data variables:
    hotdays      (time, y, x) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Attributes: (12/30)
    activity_id:           ScenarioMIP
    climate_data_id:       CMIP6
    cmip6-ng:              Some daily data has been acquired through the CMIP...
    cmor_version:          3.4.0
    Conventions:           CF-1.7 CMIP-6.2
    creation_date:         2021-11-05T01:00:00Z
    ...                    ...
    parent_activity_id:    CMIP and ScenarioMIP
    parent_experiment_id:  piControl
    parent_mip_era:        CMIP6
    parent_time_units:     days since 1850-01-01
    realm:                 atmos
    table_id:              day

In [16]:
# set hot days as coordinate reference
ref_da = hotdays.copy()
ref_da

<xarray.Dataset>
Dimensions:      (time: 1, x: 361, y: 181)
Coordinates:
  * time         (time) object 2020-01-01 12:00:00
  * x            (x) float64 -180.0 -179.0 -178.0 -177.0 ... 178.0 179.0 180.0
  * y            (y) float64 90.0 89.0 88.0 87.0 ... -87.0 -88.0 -89.0 -90.0
    spatial_ref  int64 0
Data variables:
    hotdays      (time, y, x) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Attributes: (12/30)
    activity_id:           ScenarioMIP
    climate_data_id:       CMIP6
    cmip6-ng:              Some daily data has been acquired through the CMIP...
    cmor_version:          3.4.0
    Conventions:           CF-1.7 CMIP-6.2
    creation_date:         2021-11-05T01:00:00Z
    ...                    ...
    parent_activity_id:    CMIP and ScenarioMIP
    parent_experiment_id:  piControl
    parent_mip_era:        CMIP6
    parent_time_units:     days since 1850-01-01
    realm:                 atmos
    table_id:              day

## Process GDP data

1. Gridded global datasets for Gross Domestic Product and Human Development Index over 1990-2015
2. GDP_PPP_30arcsec_v3.nc
3. https://datadryad.org/stash/dataset/doi:10.5061/dryad.dk1j0

### Read in raw gdp data 

In [17]:
gdp_raw = rioxarray.open_rasterio(path_gdp)
display(gdp_raw)
print_summary(gdp_raw)

<xarray.DataArray 'GDP_PPP' (time: 3, y: 21600, x: 43200)>
[2799360000 values with dtype=float32]
Coordinates:
  * time         (time) float64 1.99e+03 2e+03 2.015e+03
  * x            (x) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
  * y            (y) float64 90.0 89.99 89.98 89.97 ... -89.98 -89.99 -90.0
    spatial_ref  int64 0
Attributes:
    long_name:      Gross Domestic Production (GDP) (PPP)
    missing_value:  -9
    units:          ('constant 2011 international US dollar', 'constant 2011 ...
    _FillValue:     -9.0
    scale_factor:   1.0
    add_offset:     0.0

shape: (21600, 43200)
resolution: (0.008333333380429452, -0.00833333342752775)
coordinates boundary: (-180.00000101727616, -90.00000101729967, 180.00000101727616, 90.0000010172997)
CRS: None


### Clean gdp variables and dimensions

In [18]:
# select year 2015 gdp 
gdp = gdp_raw.isel(time=2)
# drop extra coordinates dimensions beyond latitude and longtitude
gdp = gdp.reset_coords(names=['time'],drop=True)
# specify CRS
gdp = gdp.rio.write_crs("epsg:4326", inplace=True)

gdp

<xarray.DataArray 'GDP_PPP' (y: 21600, x: 43200)>
[933120000 values with dtype=float32]
Coordinates:
  * x            (x) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
  * y            (y) float64 90.0 89.99 89.98 89.97 ... -89.98 -89.99 -90.0
    spatial_ref  int64 0
Attributes:
    long_name:      Gross Domestic Production (GDP) (PPP)
    missing_value:  -9
    units:          ('constant 2011 international US dollar', 'constant 2011 ...
    _FillValue:     -9.0
    scale_factor:   1.0
    add_offset:     0.0

### Aggregate GDP to 100km level

In [19]:
# aggregate GDP
gdp = xarray_square_aggregate(gdp,stride=100,
                             var_name="GDP_PPP", 
                             mode="sum")

### Reproject gdp to the refrence coordinates

In [20]:
gdp = grid_reproject(gdp, ref_da)
gdp.name = 'GDP_PPP'
gdp

<xarray.DataArray 'GDP_PPP' (y: 181, x: 361)>
array([[-90000., -90000., -90000., ..., -90000., -90000., -90000.],
       [-90000., -90000., -90000., ..., -90000., -90000., -90000.],
       [-90000., -90000., -90000., ..., -90000., -90000., -90000.],
       ...,
       [-90000., -90000., -90000., ..., -90000., -90000., -90000.],
       [-90000., -90000., -90000., ..., -90000., -90000., -90000.],
       [-90000., -90000., -90000., ..., -90000., -90000., -90000.]],
      dtype=float32)
Coordinates:
  * x            (x) float64 -180.0 -179.0 -178.0 -177.0 ... 178.0 179.0 180.0
  * y            (y) float64 90.0 89.0 88.0 87.0 ... -87.0 -88.0 -89.0 -90.0
    spatial_ref  int64 0
Attributes:
    _FillValue:  nan

## Population

### Read in population data

In [21]:
pop_raw = rioxarray.open_rasterio(path_pop_count)
display(pop_raw)
print_summary(pop_raw)

<xarray.DataArray (band: 1, y: 2983, x: 2586)>
[7714038 values with dtype=float32]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 16.46 16.47 16.48 16.49 ... 37.98 37.99 37.99 38.0
  * y            (y) float64 -22.13 -22.14 -22.15 ... -46.96 -46.97 -46.98
    spatial_ref  int64 0
Attributes:
    STATISTICS_MAXIMUM:  52822.44921875
    STATISTICS_MEAN:     33.824253836407
    STATISTICS_MINIMUM:  0
    STATISTICS_STDDEV:   339.48023586116
    _FillValue:          -99999.0
    scale_factor:        1.0
    add_offset:          0.0

shape: (2983, 2586)
resolution: (0.0083333333, -0.0083333333)
coordinates boundary: (16.457083269328677, -46.983749590994, 38.00708318312868, -22.125416357093997)
CRS: EPSG:4326


## Clean variables and dimensions

### Select band index at 0 

In [22]:
pop = pop_raw.isel(band=0)
pop

<xarray.DataArray (y: 2983, x: 2586)>
[7714038 values with dtype=float32]
Coordinates:
    band         int64 1
  * x            (x) float64 16.46 16.47 16.48 16.49 ... 37.98 37.99 37.99 38.0
  * y            (y) float64 -22.13 -22.14 -22.15 ... -46.96 -46.97 -46.98
    spatial_ref  int64 0
Attributes:
    STATISTICS_MAXIMUM:  52822.44921875
    STATISTICS_MEAN:     33.824253836407
    STATISTICS_MINIMUM:  0
    STATISTICS_STDDEV:   339.48023586116
    _FillValue:          -99999.0
    scale_factor:        1.0
    add_offset:          0.0

### Drop extra coordinates dimensions beyond latitude and longtitude


In [23]:
pop = pop.reset_coords(names=['band'],drop=True)

pop = xarray_square_aggregate(pop,stride=100,
                             var_name="population", 
                             mode="sum")

### Reproject the population to reference coordinates

In [24]:
pop = grid_reproject(pop, ref_da)
pop.name = 'population'
display(pop)

<xarray.DataArray 'population' (y: 181, x: 361)>
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)
Coordinates:
  * x            (x) float64 -180.0 -179.0 -178.0 -177.0 ... 178.0 179.0 180.0
  * y            (y) float64 90.0 89.0 88.0 87.0 ... -87.0 -88.0 -89.0 -90.0
    spatial_ref  int64 0
Attributes:
    _FillValue:  nan

## Merge number of hot days, GDP, population into one dataset

In [25]:
merge_ds = xr.merge([hotdays, gdp, pop])
display(merge_ds)
print_summary(merge_ds) 

<xarray.Dataset>
Dimensions:      (time: 1, x: 361, y: 181)
Coordinates:
  * time         (time) object 2020-01-01 12:00:00
  * x            (x) float64 -180.0 -179.0 -178.0 -177.0 ... 178.0 179.0 180.0
  * y            (y) float64 90.0 89.0 88.0 87.0 ... -87.0 -88.0 -89.0 -90.0
    spatial_ref  int64 0
Data variables:
    hotdays      (time, y, x) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    GDP_PPP      (y, x) float32 -9e+04 -9e+04 -9e+04 ... -9e+04 -9e+04 -9e+04
    population   (y, x) float32 nan nan nan nan nan nan ... nan nan nan nan nan
Attributes: (12/30)
    activity_id:           ScenarioMIP
    climate_data_id:       CMIP6
    cmip6-ng:              Some daily data has been acquired through the CMIP...
    cmor_version:          3.4.0
    Conventions:           CF-1.7 CMIP-6.2
    creation_date:         2021-11-05T01:00:00Z
    ...                    ...
    parent_activity_id:    CMIP and ScenarioMIP
    parent_experiment_id:  piControl
    parent_mip_era:        CMIP6
    parent_time_units:     days since 1850-01-01
    realm:                 atmos
    table_id:              day

shape: (181, 361)
resolution: (1.0, -1.0)
coordinates boundary: (-180.5, -90.5, 180.5, 90.5)
CRS: EPSG:4326


In [27]:
merge_df = merge_ds.to_dataframe().reset_index()
merge_df

,time,x,y,hotdays,spatial_ref,GDP_PPP,population
0,2020-01-01 12:00:00,-180.0,90.0,0.0,0,-90000.0,NaN
1,2020-01-01 12:00:00,-180.0,89.0,0.0,0,-90000.0,NaN
2,2020-01-01 12:00:00,-180.0,88.0,0.0,0,-90000.0,NaN
3,2020-01-01 12:00:00,-180.0,87.0,0.0,0,-90000.0,NaN
4,2020-01-01 12:00:00,-180.0,86.0,0.0,0,-90000.0,NaN
...,...,...,...,...,...,...,...
65336,2020-01-01 12:00:00,180.0,-86.0,0.0,0,-90000.0,NaN
65337,2020-01-01 12:00:00,180.0,-87.0,0.0,0,-90000.0,NaN
65338,2020-01-01 12:00:00,180.0,-88.0,0.0,0,-90000.0,NaN
65339,2020-01-01 12:00:00,180.0,-89.0,0.0,0,-90000.0,NaN


In [32]:
merge_df[merge_df["GDP_PPP"]<0]["GDP_PPP"].value_counts()

-90000.000000    51929
-88375.039062        2
-33469.550781        2
-53383.863281        1
-39663.652344        1
                 ...  
-9837.017578         1
-65319.695312        1
-36742.746094        1
-78200.148438        1
-83962.109375        1
Name: GDP_PPP, Length: 451, dtype: int64

In [31]:
merge_df[merge_df["population"]<0]["population"].value_counts()

-9.999889e+08    359
-3.332745e+05      1
-5.808296e+05      1
-3.136212e+05      1
-7.587158e+08      1
                ... 
-1.976788e+05      1
-4.591928e+08      1
-2.492411e+08      1
-1.795636e+05      1
-8.348379e+08      1
Name: population, Length: 100, dtype: int64

In [39]:
merge_df[(merge_df["population"].isnull()) & (merge_df["GDP_PPP"]>0)]["GDP_PPP"].value_counts()

1.348356e+03    11
1.489175e+03     6
4.690942e+05     5
1.080142e+04     5
7.736353e+05     4
                ..
2.204049e+09     1
4.945868e+08     1
2.445347e+09     1
1.376766e+09     1
2.292794e+06     1
Name: GDP_PPP, Length: 12708, dtype: int64

In [33]:
merge_df.isnull().sum()

time               0
x                  0
y                  0
hotdays            0
spatial_ref        0
GDP_PPP            0
population     64837
dtype: int64

In [28]:
# drop observations with missing population or missing GDP [note: missing was coded as -99999]
merge_df2 = (
    merge_df[(merge_df["population"] > 0) & (merge_df["GDP_PPP"]>0)]
    .copy()
    .reset_index(drop=True)
)
merge_df2

,time,x,y,hotdays,spatial_ref,GDP_PPP,population
0,2020-01-01 12:00:00,19.0,-31.0,10.750000,0,4.303034e+08,3.444283e+03
1,2020-01-01 12:00:00,19.0,-32.0,9.850000,0,4.910612e+08,1.422335e+04
2,2020-01-01 12:00:00,20.0,-29.0,22.549999,0,9.828444e+07,4.993299e+02
3,2020-01-01 12:00:00,20.0,-30.0,19.049999,0,9.507204e+05,5.468022e+02
4,2020-01-01 12:00:00,20.0,-32.0,11.900000,0,3.567785e+07,1.978308e+03
5,2020-01-01 12:00:00,20.0,-33.0,6.600000,0,7.676858e+08,4.659091e+04
6,2020-01-01 12:00:00,21.0,-28.0,22.250000,0,7.266548e+06,1.283414e+05
7,2020-01-01 12:00:00,21.0,-29.0,24.100000,0,1.440182e+09,9.345287e+03
8,2020-01-01 12:00:00,21.0,-31.0,11.200000,0,5.969609e+07,6.751965e+02
9,2020-01-01 12:00:00,21.0,-32.0,8.300000,0,3.406111e+07,4.233934e+03


In [29]:

# generate GDP PC
merge_df2["pc"] = merge_df2["GDP_PPP"] / merge_df2["population"]

# rename x and y to longitude and latitude
merge_df2.rename({"x": "Long", "y": "Lat"}, axis="columns", inplace=True)

# drop spatial ref and band
merge_df2.drop(["spatial_ref"], axis="columns", inplace=True)

# take a look at the data
merge_df2


,time,Long,Lat,hotdays,GDP_PPP,population,pc
0,2020-01-01 12:00:00,19.0,-31.0,10.750000,4.303034e+08,3.444283e+03,124932.656250
1,2020-01-01 12:00:00,19.0,-32.0,9.850000,4.910612e+08,1.422335e+04,34524.996094
2,2020-01-01 12:00:00,20.0,-29.0,22.549999,9.828444e+07,4.993299e+02,196832.671875
3,2020-01-01 12:00:00,20.0,-30.0,19.049999,9.507204e+05,5.468022e+02,1738.691528
4,2020-01-01 12:00:00,20.0,-32.0,11.900000,3.567785e+07,1.978308e+03,18034.527344
5,2020-01-01 12:00:00,20.0,-33.0,6.600000,7.676858e+08,4.659091e+04,16477.158203
6,2020-01-01 12:00:00,21.0,-28.0,22.250000,7.266548e+06,1.283414e+05,56.618877
7,2020-01-01 12:00:00,21.0,-29.0,24.100000,1.440182e+09,9.345287e+03,154107.812500
8,2020-01-01 12:00:00,21.0,-31.0,11.200000,5.969609e+07,6.751965e+02,88412.921875
9,2020-01-01 12:00:00,21.0,-32.0,8.300000,3.406111e+07,4.233934e+03,8044.789551


In [30]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)
merge_df2.describe()

,Long,Lat,hotdays,GDP_PPP,population,pc
count,46.00000,46.00000,46.00000,46.00000,46.00000,46.00000
mean,24.76087,-28.76087,13.31522,5168781824.00000,322794.62500,29059.35938
std,3.66476,3.01982,7.02993,19374045184.00000,853288.75000,42196.41016
min,19.00000,-33.00000,0.25000,950720.43750,499.32990,56.61888
25%,22.00000,-31.00000,8.26250,113049102.00000,9783.41650,5925.80725
50%,24.00000,-29.00000,12.15000,791640864.00000,44064.22656,12494.44629
75%,28.00000,-27.00000,19.23750,2772212928.00000,374923.45312,31498.40820
max,31.00000,-23.00000,24.90000,130714714112.00000,5647802.00000,196832.67188


### Get percentiles for mapping

GDP PC percentiles - bottom 40%, 40-80%, 80-95%, above 95%

In [39]:
gdp_pc_values = merge_df2['pc']
gdp_pc_cutoff = np.nanquantile(gdp_pc_values,[0.4, 0.8, 0.95, 1])

# turn off scientific notation in numpy arrays
np.set_printoptions(formatter={'float': '{:0.4f}'.format})
print(gdp_pc_cutoff)

[10394.1750 46228.4672 232189.8328 9298811904.0000]


Population percentiles - bottom 10%, 10-50%, 50-75%, 75-90%, 90-95%, 95-99%, above 99%

In [40]:
pop_values = merge_df2['population']
pop_cutoff = np.nanquantile(pop_values,[0.1, 0.5, 0.75, 0.9, 0.95, 0.99, 1])
pop_cutoff

array([0.2487, 7.8072, 59.6036, 315.9083, 833.0137, 3540.1640, 52822.4492])

# Data Analysis

## Number of hot days

In [43]:
df = pd.read_csv(os.path.join(path_map, "hd35_climatology_annual-monthly_cmip6_ssp119_anomaly_ensemble_2020-2039_median_ZAF.csv"), header=1)
df

,Unnamed: 0,Annual,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,South Africa,11.39,2.00,1.68,0.85,0.21,0.01,0.0,0.0,0.04,0.52,1.86,2.13,2.28
1,Eastern Cape,3.45,0.90,0.53,0.17,0.07,0.00,0.0,0.0,0.00,0.02,0.32,0.59,1.01
2,KwaZulu-Natal,2.10,0.38,0.18,0.12,0.00,-0.00,0.0,0.0,-0.00,0.06,0.39,0.39,0.52
3,Limpopo,15.36,1.35,1.10,0.74,0.38,0.03,0.0,0.0,0.17,1.03,4.07,3.15,1.63
4,Mpumalanga,3.90,0.28,0.26,0.09,0.01,0.00,0.0,0.0,0.01,0.17,1.20,0.97,0.72
5,North West,17.96,3.28,2.46,1.58,0.20,0.00,0.0,0.0,0.02,1.25,4.45,4.13,3.47
6,Northern Cape,15.60,2.93,2.87,1.42,0.31,0.00,0.0,0.0,0.01,0.48,1.52,2.47,3.16
7,Gauteng,10.03,1.22,0.77,0.28,0.00,0.00,0.0,0.0,0.00,0.50,2.67,2.35,1.69
8,Free State,12.16,2.47,1.73,0.51,0.02,0.00,0.0,0.0,0.00,0.29,1.87,2.78,3.79
9,Western Cape,2.99,0.92,0.86,0.40,0.04,0.00,0.0,0.0,0.00,-0.00,0.11,0.34,0.56


## GDP rating

In [41]:
# # assign gdp rating 
# gdp_values = merge_df2['GDP_PPP']
# gdp_cutoff = np.nanquantile(gdp_values,[0.7,0.95])
# gdp_levels = np.array(['nan']*len(gdp_values))
# gdp_levels[~np.isnan(gdp_values)] = 'C'
# gdp_levels[gdp_values > gdp_cutoff[0]] = 'B'
# gdp_levels[gdp_values > gdp_cutoff[1]] = 'A'
# merge_df2['gdp_rating'] = gdp_levels
# merge_df2['gdp_rating'].value_counts()

In [42]:
# %%time
# merge_df2 = merge_df2.rename({'x':'Long','y':'Lat'},axis=1)
# if read in age sex csv, there will be some decimal rounding difference due to float type
# merge_df2['Lat_5_decimal'] = np.round(merge_df2['Lat'],5).astype('str')
# merge_df2['Long_5_decimal'] = np.round(merge_df2['Long'],5).astype('str')
# display(merge_df2.head())


# Save the final data

In [31]:
%%time
merge_df2.to_csv(os.path.join(path_data_out, "zaf_GDP_hotdays_100km.csv"))

CPU times: user 3.74 ms, sys: 5.04 ms, total: 8.78 ms
Wall time: 18.4 ms


# Total runtime

In [44]:
end_time = time.perf_counter ()
print(round((end_time - start_time)/60,2), "minutes")

0.99 minutes
